Recommendation System

In [94]:
import warnings
warnings.filterwarnings('ignore')
from collections import Counter

# Import necessary packages
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

In [95]:
# Load Dataset anime.csv
df = pd.read_csv("anime.csv")

In [96]:
df.shape

(12294, 7)

In [97]:
df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [98]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


In [99]:
len(df.genre.unique())

3265

In [100]:
len(df.rating.unique())

599

In [101]:
len(df.name.unique())

12292

In [102]:
df.rating.unique()

array([ 9.37,  9.26,  9.25,  9.17,  9.16,  9.15,  9.13,  9.11,  9.1 ,
        9.06,  9.05,  9.04,  8.98,  8.93,  8.92,  8.88,  8.84,  8.83,
        8.82,  8.81,  8.8 ,  8.78,  8.77,  8.76,  8.75,  8.74,  8.73,
        8.72,  8.71,  8.69,  8.68,  8.67,  8.66,  8.65,  8.64,  8.62,
        8.61,  8.6 ,  8.59,  8.58,  8.57,  8.56,  8.55,  8.54,  8.53,
        8.52,  8.51,  8.5 ,  8.49,  8.48,  8.47,  8.46,  8.45,  8.44,
        8.43,  8.42,  8.41,  8.4 ,  8.39,  8.38,  8.37,  8.36,  8.35,
        8.34,  8.33,  8.32,  8.31,  8.3 ,  8.29,  8.28,  8.27,  8.26,
        8.25,  8.24,  8.23,  8.22,  8.21,  8.2 ,  8.19,  8.18,  8.17,
        8.16,  8.15,  8.14,  8.13,  8.12,  8.11,  8.1 ,  8.09,  8.08,
        8.07,  8.06,  8.05,  8.04,  8.03,  8.02,  8.01,  8.  ,  7.99,
        7.98,  7.97,  7.96,  7.95,  7.94,  7.93,  7.92,  7.91,  7.9 ,
        7.89,  7.88,  7.87,  7.86,  7.85,  7.84,  7.83,  7.82,  7.81,
        7.8 ,  7.79,  7.78,  7.77,  7.76,  7.75,  7.74,  7.73,  7.72,
        7.71,  7.7 ,

In [103]:
df.groupby('name')['rating'].mean().sort_values(ascending=False)[:6]

,rating
name,
Taka no Tsume 8: Yoshida-kun no X-Files,10.00
Spoon-hime no Swing Kitchen,9.60
Mogura no Motoro,9.50
Kimi no Na wa.,9.37
Kahei no Umi,9.33
Fullmetal Alchemist: Brotherhood,9.26


In [104]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
anime_id,12294.0,14058.221653,11455.294701,1.00,3484.25,10260.50,24794.50,34527.0
rating,12064.0,6.473902,1.026746,1.67,5.88,6.57,7.18,10.0
members,12294.0,18071.338864,54820.676925,5.00,225.00,1550.00,9437.00,1013917.0


In [105]:
df.isnull().sum()

,0
anime_id,0
name,0
genre,62
type,25
episodes,0
rating,230
members,0


In [106]:
# handling missing values
df = df.dropna(subset=["genre"])
df["rating"] = df["rating"].fillna(df["rating"].median())
df["type"] = df["type"].fillna(df["type"].mode()[0])
df['episodes'] = df['episodes'].replace({'Unknown': 0, '?': 0}).astype(int)  # Handle 'episodes' column: Replace 'Unknown' or '?' with 0 and convert to numeric

Feature Selection (Genre-Based)

anime genre as primary similarity feature.

In [107]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity,euclidean_distances

MultiLabelBinarizer converts multiple labels per record into a binary (0/1) matrix

cosine_similarity -- how similar two vectors are based on angle, not magnitude.

euclidean_distance -- straight-line distance between two points.

# Cosine_Similarity:

Item based

In [108]:
# en_out_hot = df["genre"].str.get_dummies(sep = ", ")

In [109]:
# Concatenate the one-hot encoded genres with the original dataframe
# anime_df_clean = pd.concat([df, en_out_hot], axis=1)

# Drop unnecessary columns ('anime_id', 'name', 'genre', 'type')
# anime_df_clean = anime_df_clean.drop(columns=['anime_id', 'name', 'genre', 'type'])

In [110]:
# Normalize numerical features (rating and members)
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# anime_df_clean[['rating', 'members']] = scaler.fit_transform(anime_df_clean[['rating', 'members']])

# print(anime_df_clean.head())

In [111]:
df["genre_list"] = df["genre"].str.split(", ")
df["genre_list"]

,genre_list
0,"[Drama, Romance, School, Supernatural]"
1,"[Action, Adventure, Drama, Fantasy, Magic, Mil..."
2,"[Action, Comedy, Historical, Parody, Samurai, ..."
3,"[Sci-Fi, Thriller]"
4,"[Action, Comedy, Historical, Parody, Samurai, ..."
...,...
12289,[Hentai]
12290,[Hentai]
12291,[Hentai]
12292,[Hentai]


In [112]:
mlb = MultiLabelBinarizer()
item_feature_matrix = mlb.fit_transform(df["genre_list"])
item_feature_matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [113]:
item_item_similarity = cosine_similarity(item_feature_df)
item_item_similarity

array([[1.        , 0.18898224, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.18898224, 1.        , 0.28571429, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.28571429, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 1.        ,
        1.        ],
       [0.        , 0.        , 0.        , ..., 1.        , 1.        ,
        1.        ],
       [0.        , 0.        , 0.        , ..., 1.        , 1.        ,
        1.        ]])

In [114]:
item_item_sim_df = pd.DataFrame(
    item_item_similarity,
    index=item_feature_df.index,
    columns=item_feature_df.index
)
item_item_sim_df

name,Kimi no Na wa.,Fullmetal Alchemist: Brotherhood,Gintama°,Steins;Gate,Gintama&#039;,Haikyuu!!: Karasuno Koukou VS Shiratorizawa Gakuen Koukou,Hunter x Hunter (2011),Ginga Eiyuu Densetsu,Gintama Movie: Kanketsu-hen - Yorozuya yo Eien Nare,Gintama&#039;: Enchousen,...,Super Erotic Anime,Taimanin Asagi 3,Teleclub no Himitsu,Tenshi no Habataki Jun,The Satisfaction,Toushindai My Lover: Minami tai Mecha-Minami,Under World,Violence Gekiga David no Hoshi,Violence Gekiga Shin David no Hoshi: Inma Densetsu,Yasuji no Pornorama: Yacchimae!!
name,,,,,,,,,,,,,,,,,,,,,
Kimi no Na wa.,1.000000,0.188982,0.000000,0.000000,0.000000,0.447214,0.000000,0.250000,0.000000,0.000000,...,0.0,0.288675,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Fullmetal Alchemist: Brotherhood,0.188982,1.000000,0.285714,0.000000,0.285714,0.338062,0.566947,0.377964,0.285714,0.285714,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Gintama°,0.000000,0.285714,1.000000,0.267261,1.000000,0.338062,0.377964,0.188982,1.000000,1.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Steins;Gate,0.000000,0.000000,0.267261,1.000000,0.267261,0.000000,0.000000,0.353553,0.267261,0.267261,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Gintama&#039;,0.000000,0.285714,1.000000,0.267261,1.000000,0.338062,0.377964,0.188982,1.000000,1.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Toushindai My Lover: Minami tai Mecha-Minami,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,0.577350,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Under World,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,0.577350,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Violence Gekiga David no Hoshi,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,0.577350,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [115]:
def recommend_similar_anime(anime_name, top_n=10):
    if anime_name in item_item_sim_df.columns:
        return (
            item_item_sim_df[anime_name]
            .sort_values(ascending=False)
            .iloc[1:top_n+1]
        )
    else:
        print("Anime is not in the list")


In [117]:
recommend_similar_anime("Naruto")
# more recommended are...type name and get
# future obj: use embedded nlp

,Naruto
name,
Naruto: Shippuuden Movie 3 - Hi no Ishi wo Tsugu Mono,1.000000
Boruto: Naruto the Movie,1.000000
Naruto x UT,1.000000
Boruto: Naruto the Movie - Naruto ga Hokage ni Natta Hi,1.000000
Naruto Soyokazeden Movie: Naruto to Mashin to Mitsu no Onegai Dattebayo!!,1.000000
Naruto: Shippuuden Movie 4 - The Lost Tower,1.000000
Naruto,1.000000
Naruto: Shippuuden,1.000000
Kyutai Panic Adventure!,0.894427


Evaluation

## Wrong check again

In [139]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

In [126]:
# Split the dataset into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [134]:
# Function to get positive recommendations based on a genre
def is_relevant(g1, g2):
    return len(set(g1.split(", ")) & set(g2.split(", "))) > 0

In [132]:
y_true, y_pred = [], []

for anime in test_df["name"].sample(50, random_state=42):
    try:
        target_genres = set(
            df[df["name"] == anime]["genre"].values[0].split(", ")
        )

        recs = recommend_similar_anime(anime, df, df["genre_list"])

        for _, row in recs.iterrows():
            rec_genres = set(row["genre"].split(", "))

            y_true.append(1)
            y_pred.append(1 if target_genres & rec_genres else 0)

    except:
        continue


In [133]:
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

precision, recall, f1

(0.0, 0.0, 0.0)